In [93]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
from scipy.optimize import fmin_slsqp
import numpy as np
import matplotlib.pyplot as plt
from time import time
import os
import sys
from pathlib import Path
ROOT_PATH = Path().absolute().parent
if str(ROOT_PATH) not in sys.path:
    sys.path.insert(1, str(ROOT_PATH))
from frequency_response import FrequencyResponse
from biquad import peaking, digital_coeffs

In [3]:
fr = FrequencyResponse.read_from_csv(ROOT_PATH.joinpath('results', 'oratory1990', 'harman_over-ear_2018', 'Audeze LCD-4', 'Audeze LCD-4.csv'))

In [95]:
def print_filters(filters):
    print(', '.join([f'{fc:.0f} Hz' for fc in filters[:, 0]]))
    print(', '.join([f'{q:.2f} Q' for q in filters[:, 1]]))
    print(', '.join([f'{gain:.1f} dB' for gain in filters[:, 2]]))
    #for fc, q, gain in filters:
    #    print(f'{fc:.0f} Hz, {q:.3f} Q, {gain:.1f} dB')

In [115]:
def compare(fp, n):
    fr = FrequencyResponse.read_from_csv(fp)
    old_fbeq = fr.fixed_band_eq.copy()
    old_peq = fr.parametric_eq.copy()
    
    t = time()
    #fr.optimize_parametric_eq(max_filters=n)
    #fr.optimize_fixed_band_eq([31.25, 62.5, 125, 250, 500, 1000, 2000, 4000, 8000, 16000], [1.41] * 10, fs=48000)
    #fr.parametric_eq = fr.fixed_band_eq.copy()
    old_time = time() - t

    t = time()
    #filters, n_filters, max_gains = fr.optimize_peq(n, fs=48000)
    filters, n_filters, max_gain = fr.optimize_fbeq([31.25, 62.5, 125, 250, 500, 1000, 2000, 4000, 8000, 16000], 1.41, fs=48000)
    #fr.fixed_band_eq = fr.parametric_eq.copy()
    new_time = time() - t
    
    fr.parametric_eq = old_peq

    ix = np.sum(fr.frequency < 10e3)
    new_rmse = np.sqrt(np.mean(np.square(fr.equalization[:ix] - fr.fixed_band_eq[:ix])))
    old_rmse = np.sqrt(np.mean(np.square(fr.equalization[:ix] - fr.parametric_eq[:ix])))
    print(f'{fr.name}: RMSE {old_rmse:.2f} -> {new_rmse:.2f}  = {"+" if new_rmse > old_rmse else ""}{new_rmse - old_rmse:.2f} dB ({(new_rmse - old_rmse) / old_rmse * 100:.1f} %) @ {old_time*1000:.0f} -> {new_time*1000:.0f} ms')
    fig, axs = plt.subplots(1, 2)
    fig.set_size_inches(26, 8)
    FrequencyResponse.init_plot(fig=fig, ax=axs[1])
    fr.plot_graph(fig=fig, ax=axs[0], fixed_band_eq_plot_kwargs={'label': 'New optimizer'}, show=False)
    for fc, q, gain in filters:
        axs[1].plot(fr.frequency, digital_coeffs(fr.frequency, 48e3, *peaking(fc, q, gain, fs=48e3), reduce=True), label=f'{fc:.0f} Hz, {q:.3f} Q, {gain:.1f} dB')
    axs[1].legend()
    axs[1].set_title('Filters')
    plt.show()

In [ ]:
n = [5, 5]
i = 0
for fp in ROOT_PATH.joinpath('results', 'oratory1990').glob('*/*/*.csv'):
    compare(fp, n)
    i += 1
    if i > 10:
        break